# (day2) Section 1: 勾配消失問題

本書は、「深層学習前編（day2）レポート」の、「Section 1: 勾配消失問題」についてのものです。


## 1. 要点まとめ

勾配消失問題は、学習が進むと、勾配が小さくなり、パラメータが更新されないこと。結果、訓練で大域的局所解に収束しない。


シグモイド関数は、大きな値の場合、変化が小さくなる。それで、勾配(変化) が消失してしまう。


勾配消失を解決する方法は、以下の通り。

- 活性化関数

- 重みの初期値

- バッチ正規化


### 活性化関数

活性化関数をシグモイド関数から、 ReLU 関数に変える。

$$
\begin{equation}
\begin{cases}
x & ( x \gt 0 ) \\
0 & ( x \leqq 0 )
\end{cases}
\end{equation}
$$


- NOTE: スパース化<br>
  以下を参考にした。<br>
  - [東芝レビュー74巻4号（2019年7月）](https://www.global.toshiba/content/dam/toshiba/migration/corp/techReviewAssets/tech/review/2019/04/74_04pdf/f01.pdf)<br>
  > スパース（疎）はまばらという意味 であり，機械学習や統計学の分野ではデータの大半がゼロ で，意味のあるものは少数に限られること<br>
  ...<br>
  > ReLUは入力が 正の場合は恒等写像を，ゼロ以下の場合はゼロを出力する 関数である<br>

上記の、 "ゼロ以下の場合はゼロを出力する" が、意味のあるものを残すスパース化に貢献していると理解した。


### 初期値の設定方法

NN の「個性」を与えるため、重み ( $ \textbf{W} $ ) の初期値の与え方の分布を調整する。<br>
重みの分布が 0, 1 に偏ったり、中央値に集中すると、 NN としての学習の意味がないため、正規分布の乱数で初期値を与える。


#### Xavier

```python
    # Xavierの初期値
    network['W1'] = np.random.randn(input_layer_size, hidden_layer_1_size) / (np.sqrt(input_layer_size))
```

- [numpy.random.randn — NumPy v1.21 Manual](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html)<br>
  > Return a sample (or samples) from the “standard normal” distribution.

正規分布で発生させた乱数値を、前層ノード数の平方根で除算。

- NOTE: [Xavierの初期値](https://qiita.com/Becon147/items/a9971041bca5c10483bc#xavier%E3%81%AE%E5%88%9D%E6%9C%9F%E5%80%A4)<br>
  > 平均0,標準偏差 $ \frac{1}{\sqrt{n}} $ である正規分布から初期値を生成


#### He

```python
    # Heの初期値
    network['W1'] = np.random.randn(input_layer_size, hidden_layer_1_size) / np.sqrt(input_layer_size) * np.sqrt(2)
```

Xavier に $ \sqrt{2} $ を掛けたもの。

- NOTE: [ニューラルネットワークの重みの初期値について解説! | AVILEN AI Trend](https://ai-trend.jp/basic-study/neural-network/initial_value/)<br>
  > ReLU関数を活性化関数としたモデルにXavierの初期値を用いるとアクティベーション分布に偏りが生じやすくなる


### バッチ正規化


#### ミニバッチ単位

特にイメージデータの場合、バッチ全体のデータで一度に学習することができない。(メモリー量大)
ミニバッチ単位に訓練データを分割する

`2_3_batch_normalization.ipynb` の中で対応するコードは以下の通り。訓練データからランダムに 100 件を抽出している。

```python
batch_size = 100
...

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    d_batch = d_train[batch_mask]
```


#### バッチ正規化層

ミニバッチ単位で、データの分布を正規化する。
学習データの極端な偏りをなくし、過学習を回避できる。


#### バッチ正規化の数式

最終的に求めたいのは、「4. 変倍・移動」である。

ミニバッチ単位で、訓練データ $ x $ の正規化 ( 標準化 ) を行った値で、<br>
出力 $ y $ を「変倍・移動」する。


##### 1. ミニバッチの平均

$$
\mu_t = \frac{1}{N_t} \sum_{i=1}^{N_t} x_{ni}
$$

ミニバッチ単位に含まれる訓練データが、 8 枚の場合、 $ N_t = 8 $ 。


##### 2. ミニバッチの分散

$$
\sigma_t^2 = \frac{1}{N_t} \sum_{i=1}^{N_t} (
  x_{ni} - \mu_t
)^2
$$


##### 3. ミニバッチの正規化

ミニバッチ単位の中で、正規化 ( 標準化 ) された値を得る。

$$
\hat{x_{ni}} = \frac{x_{ni} - \mu_t}
{\sqrt{\sigma_t^2 + \theta}}
$$

$ \theta $ はバイアス。


- NOTE: 統計的正規化
  - [正規化（Normalization）／標準化（Standardization）とは？：AI・機械学習の用語辞典 - ＠IT](https://atmarkit.itmedia.co.jp/ait/articles/2110/07/news027.html)<br>
    > 平均0、分散1にスケーリングする「Z-score normalization」<br>
    ...<br>
    > Z-score normalizationは、標準化（Standardization）と呼ばれる

ここでは、「標準化」 ( 平均0、分散1にスケーリング ) をしている。


##### 4. 変倍・移動

NN が扱いやすいように、定数倍 ( $ \gamma $ )、バイアス ( $ \beta $ ) を加える。

$$
y_{ni} = \gamma x_{ni} + \beta
$$


対応するコードは、以下の通り。

- common/layers.py: `class BatchNormalization`<br>
  ```pyhon
    def __forward(self, x, train_flg):
...
            mu = x.mean(axis=0)
            xc = x - mu
            var = np.mean(xc**2, axis=0)
            std = np.sqrt(var + 10e-7)
            xn = xc / std
...
        out = self.gamma * xn + self.beta 
        return out
    ```


## 2. 確認テスト

以降の "page. " は、講義資料のページの番号です。


### page. 10

$$
\frac{\partial{z}}{\partial{x}}
=
\frac{\partial{z}}{\partial{t}}
\frac{\partial{t}}{\partial{x}}
$$

$$
=
2t \times 1 \\
=
2(x + y)
$$


### page. 18

0.25 になる。

シグモイド関数の微分は、以下であるから。

$$
f(x) = \frac{1}{1 + e^{x}}
$$

とすると、

$$
f'(x) = f(x) (1 - f(x))
$$

$$
f(0) = 0.5
$$

から、

$$
0.5 \times (1 - 0.5) = 0.25s
$$


### page. 26

重みを 0 に設定すると、それぞれの重みの多様性が失われ、結果、 NN として意味のない重みとなってしまうため。
そこから、学習を重ねると非効率であるため。
